# Set up

In [3]:
!pip install scikit-learn==1.5.2

import numpy
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

data = pd.read_csv('/kaggle/input/frases-playlist/frases_playlist.csv')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 85.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.3 which is incompatible.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.10.2 which is incompatible.


In [4]:
data.head()

,frase
0,Quiero una playlist que me motive en el gimnasio.
1,"Me ha dejado mi pareja, quiero una playlist pa..."
2,"Voy a hacer un viaje largo, quiero una playlis..."
3,No se qué canciones poner en el coche.
4,Quiero música para meditar.


# Create dataset

In [5]:
!pip install openai
from openai import AzureOpenAI

AZURE_OPENAI_API_KEY = "7MoKzuuPAla0Cl3RYSM2aHjTAmkVopeVC4VhHCCdLPGDbAg0Vk4OJQQJ99AKACYeBjFXJ3w3AAABACOGrjrr"
API_VERSION = "2024-10-01-preview"
API_ENDPOINT = "https://practicas-2024.openai.azure.com/"
API_LOCATION = "eastus"

API_MODEL_MINI = "gpt-4o-mini"
API_MODEL = "gpt-4o"

client = AzureOpenAI(
            api_version=API_VERSION,
            azure_endpoint=API_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.2/390.2 kB 6.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 17.0 MB/s eta 0:00:00


In [6]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import re

stop_words = set(stopwords.words('spanish'))

def lower_text(text):
    return text.lower()

def remove_special_char_and_numbers(text):
    return re.sub("(\\d|\\W)+"," ",text)

def remove_extra_spaces(text):
    return " ".join(word_tokenize(text))

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

def extract_keywords(text):
    text = lower_text(text)
    text = remove_special_char_and_numbers(text)
    text = remove_extra_spaces(text)
    text = remove_stopwords(text)
    return text

data['keywords']=data['frase'].progress_apply(extract_keywords)

100%|██████████| 1005/1005 [00:00<00:00, 7026.61it/s]


In [7]:
CONTEXT="""
Por la frase que recibas debes etiquetarla con:
Pop
Rock
Hip-Hop
EDM
Indie
Jazz
Soul
Reggaetón
Música Clásica
Folk
R&B
Canciones acústicas
Lo-fi
Música instrumental
Música ambiental
Música latina
Funk
Música tropical
Bossa Nova
Dance

Separa las etiquetas con coma
"""

def receive_message(text):
        try:
            chat_completion = client.chat.completions.create(
                model=API_MODEL,
                messages=[
                    {"role": "system", "content": CONTEXT},
                    {"role": "user", "content": text},
                ],
                max_tokens=10,
                temperature=0.0,  # Reducir la aleatoriedad para respuestas consistentes
            )

            response = chat_completion.choices[0].message.content.strip()
            return response

        except:
            return "ERROR"

In [8]:
data['genero']=data['keywords'].progress_apply(receive_message)

100%|██████████| 1005/1005 [09:04<00:00,  1.85it/s] 


In [9]:
data.head()

,frase,keywords,genero
0,Quiero una playlist que me motive en el gimnasio.,quiero playlist motive gimnasio,"Dance, EDM, Hip-Hop, Pop, Reg"
1,"Me ha dejado mi pareja, quiero una playlist pa...",dejado pareja quiero playlist llorar,"R&B, Soul, Indie, Canciones acúst"
2,"Voy a hacer un viaje largo, quiero una playlis...",voy hacer viaje largo quiero playlist acompañe...,"Lo-fi, Música ambiental, Música instrumental, ..."
3,No se qué canciones poner en el coche.,canciones poner coche,"Pop, Rock, Hip-Hop, EDM, Indie"
4,Quiero música para meditar.,quiero música meditar,"Música ambiental, Música instrumental, Lo-fi"


In [10]:
genders = ["Pop", "Rock","Hip-Hop","EDM","Indie","Jazz","Soul","Reggaetón","Música Clásica","Folk","R&B","Canciones acústicas","Lo-fi",
"Música instrumental","Música ambiental","Música latina","Funk","Música tropical","Bossa Nova","Dance"]

In [11]:
for gender in genders:
    data[gender] = data['genero'].apply(lambda frase: gender in frase)

In [12]:
data.head()

,frase,keywords,genero,Pop,Rock,Hip-Hop,EDM,Indie,Jazz,Soul,...,R&B,Canciones acústicas,Lo-fi,Música instrumental,Música ambiental,Música latina,Funk,Música tropical,Bossa Nova,Dance
0,Quiero una playlist que me motive en el gimnasio.,quiero playlist motive gimnasio,"Dance, EDM, Hip-Hop, Pop, Reg",True,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,"Me ha dejado mi pareja, quiero una playlist pa...",dejado pareja quiero playlist llorar,"R&B, Soul, Indie, Canciones acúst",False,False,False,False,True,False,True,...,True,False,False,False,False,False,False,False,False,False
2,"Voy a hacer un viaje largo, quiero una playlis...",voy hacer viaje largo quiero playlist acompañe...,"Lo-fi, Música ambiental, Música instrumental, ...",False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,False,False
3,No se qué canciones poner en el coche.,canciones poner coche,"Pop, Rock, Hip-Hop, EDM, Indie",True,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Quiero música para meditar.,quiero música meditar,"Música ambiental, Música instrumental, Lo-fi",False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,False,False


# Try embbeding

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

case = 2
if case == 1:
    vectorizer = TfidfVectorizer()
elif case == 2:
    vectorizer = CountVectorizer(stop_words=stopwords.words('spanish'))
elif case == 3:
    vectorizer = CountVectorizer(stop_words=stopwords.words('spanish'), ngram_range=(1,2))

X = vectorizer.fit_transform(data['frase'])
y = data.drop(['frase','genero','keywords'], axis=1)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Try models

In [15]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

case = 1
if case == 1:
    model = LogisticRegression(solver='liblinear', max_iter=1000) #Accuracy media: 0.9176616915422885
elif case == 2:
    model = SVC(kernel='linear', probability=False) #Accuracy media: 0.9164179104477611
elif case == 3:
    model = RandomForestClassifier(n_estimators=200, random_state=42) #Accuracy media: 0.9129353233830846
elif case == 4:
    model = DecisionTreeClassifier(max_depth=5, random_state=42) #Accuracy media: 0.9024875621890548
elif case == 5:
    model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42) #Accuracy media: 0.9072139303482587
elif case == 6:
    model = KNeighborsClassifier(n_neighbors=5) #Accuracy media: 0.8885572139303483


multi_target_model = MultiOutputClassifier(model)

multi_target_model.fit(X_train, y_train)

y_pred = multi_target_model.predict(X_test)


In [16]:
from sklearn.metrics import accuracy_score

med = 0
for i, genre in enumerate(genders):  
    acc = accuracy_score(y_test.iloc[:, i], y_pred[:, i]) 
    med += acc
    print(f"Accuracy para {genre}: {acc}")

med /= len(genders)
print(f"Accuracy media: {med}")

Accuracy para Pop: 0.8606965174129353
Accuracy para Rock: 0.9104477611940298
Accuracy para Hip-Hop: 0.945273631840796
Accuracy para EDM: 0.9054726368159204
Accuracy para Indie: 0.8656716417910447
Accuracy para Jazz: 0.9253731343283582
Accuracy para Soul: 0.9751243781094527
Accuracy para Reggaetón: 0.945273631840796
Accuracy para Música Clásica: 0.9402985074626866
Accuracy para Folk: 0.9900497512437811
Accuracy para R&B: 0.9353233830845771
Accuracy para Canciones acústicas: 0.9751243781094527
Accuracy para Lo-fi: 0.8606965174129353
Accuracy para Música instrumental: 0.7810945273631841
Accuracy para Música ambiental: 0.8109452736318408
Accuracy para Música latina: 0.8955223880597015
Accuracy para Funk: 0.9800995024875622
Accuracy para Música tropical: 0.9601990049751243
Accuracy para Bossa Nova: 0.9552238805970149
Accuracy para Dance: 0.8756218905472637
Accuracy media: 0.9146766169154228


# Export

In [18]:
import joblib

joblib.dump((multi_target_model, vectorizer), 'musical_gender_model.pkl')

['musical_gender_model.pkl']